In [19]:
from tensorflow import keras, expand_dims
import numpy as np
from modules import data_feed
model_name = 'VisFin_v5.h5'


In [20]:
def make_new_model(dropout = 0):
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(9, kernel_size = (9,18),padding='same', input_shape=(201, 43, 1), activation='elu'))
    model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Conv2D(9, kernel_size = (9,18), padding='same', activation='elu'))
    model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.MaxPooling2D())
    model.add(keras.layers.Conv2D(9, kernel_size = (6,12),padding='same', activation='elu'))
    model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.MaxPooling2D())
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(420, activation='elu'))
    model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(160, activation='elu'))
    model.add(keras.layers.Dense(42, activation='elu'))
    model.add(keras.layers.Dense(7,activation='softmax'))
    model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [21]:
# Modelig data
modeling = data_feed.PrepareData('daily__us__nasdaq_stocks.csv', '<DATE>', skip=5)
modeling.clean_db()

In [22]:
try:
    VisFin = keras.models.load_model(model_name)
    print('Loading!')
except OSError:
    VisFin = make_new_model(dropout=0.1)
    print('Compiling!')

Loading!


In [23]:
# TRAINING
data = modeling.mass_hacking(tick=0, horizon=42)
tr_data = expand_dims(np.array(data[0]),axis=-1)
tr_key = np.array(data[1])
if tr_data.shape[0] > 0:
    VisFin.fit(tr_data, tr_key, epochs=3, verbose=2)
VisFin.save(model_name)


In [24]:
# # # convert to JS
# import tensorflowjs
# # tensorflowjs.converters.save_keras_model(VisFin0, 'js_format\\VisFin_0')
# # tensorflowjs.converters.save_keras_model(VisFin1, 'js_format\\VisFin_1')
# # tensorflowjs.converters.save_keras_model(VisFin2, 'js_format\\VisFin_2')
# # tensorflowjs.converters.save_keras_model(VisFin3, 'js_format\\VisFin_3')
